# How to use OpenAI function calling with LLMs
Function calling extends the power capabilities of LLMs. It allolws you to format
the output of an LLM response into a JSON object, which then can be fed down stream
to an actual function as a argument to process the response.

OpenAI documention states the basic steps involved in function calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

<img src="./images/gpt_function_calling.png">

Let's first demonstrate how we use this feature. Let's first specify a function and use the API to generate function arguments.

## How to generate function arguments


In [136]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Function calling for Anyscale Endpoints is coming soon. Not yet ready as of writing this notebook!

In [137]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


Define some utitilty functions

In [138]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [216]:
def get_commpletion(clnt: object, model: str, user_content:str, functions: object) -> object:
    chat_completion = clnt.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": user_content}],
        functions = [{
            "name": "add_prime_numbers",
            "description": "Get a list 20 integer prime numbers between 2 and 100",
            "parameters": {
                "type": "object",
                "properties": {
                    "prime_numbers": {
                        "type": "array",
                        "items": {
                            "type": "integer",
                            "description": "A list of 20 prime numbers"
                        },
                        "description": "List of of 20 prime numbers to be added"
                    }
                },
                "required": ["add_prime_numbers"]
            }
        }],
        function_call={"name": "add_prime_numbers"}
    )
    response = chat_completion.choices[0].message
    return response

In [217]:
from typing import List, Dict
def add_prime_numbers(p_numbers: Dict[str, List[int]]) -> int:
    return sum(p_numbers["prime_numbers"])

Define the functions argument and function specification as part of the message


In [218]:
user_content = "Add 20 random prime numbers between 2 and 100?"

In [219]:
print(f"Using Endpoints: {openai.api_base} ...\n")
response = get_commpletion(client, MODEL, user_content, functions)
print(response)

Using Endpoints: https://api.openai.com/v1 ...

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"prime_numbers":[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71]}', name='add_prime_numbers'), tool_calls=None)


#### Example 1: Extract the generated arguments
Let's generate arguments to compute the sum of random
prime numbers between 2 and 100. These would a list fed
into a function that can compute its sum.

The idea is here is to nudge the LLM to generate JSON object, which can be easily converted into a Python dictionary, as an  argument for a function downstream to compute the sum.

We can convert the resonse into a dictionary

In [220]:
import json

In [221]:
json_arguments = response.dict()
json_arguments

{'content': None,
 'role': 'assistant',
 'function_call': {'arguments': '{"prime_numbers":[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71]}',
  'name': 'add_prime_numbers'},
 'tool_calls': None}

In [222]:
#Extract specific items from the dict
print(f"Function name: {json_arguments['function_call']['name']}")

Function name: add_prime_numbers


In [223]:
funcs = response.dict()['function_call']['arguments']
funcs_args = json.loads(funcs)
print(f"Arguments: {funcs_args}")

Arguments: {'prime_numbers': [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]}


#### Invoke the function from within your app

In [224]:
sum_of_prime = add_prime_numbers(funcs_args)
sum_of_prime

639

#### Example 2: Extract the generated arguments


In [234]:
import folium

def create_map(path: str) -> None:
    # Create a base map
    m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)
    
    # Adding markers for each city
    folium.Marker([51.5074, -0.1278], popup='London').add_to(m)
    folium.Marker([40.7128, -74.0060], popup='New York').add_to(m)
    folium.Marker([37.7749, -122.4194], popup='San Francisco').add_to(m)

    # Display the map
    m.save(path)

In [235]:
from IPython.display import IFrame

# Assuming the HTML file is named 'example.html' and located in the same directory as the Jupyter Notebook
html_file_path = './world_map_with_cities.html'
create_map(html_file_path)

# Display the HTML file in the Jupyter Notebook
IFrame(src=html_file_path, width=700, height=400)